# MVG Rad Bike Ride Data Analysis – Munich

This analysis explores a dataset of public bike rental trips in Munich using Kotlin, DataFrame, and Kandy/Kandy-Geo.

It focuses on revealing patterns in:
- Temporal distribution of rides (by hour, month, season, weekday)
- Most used stations and common routes
        - Spatial distribution of rides across Munich districts
- Ride durations and geographical start/end points

The dataset contains geolocation data, timestamps, and station names. By combining this with a GeoJSON file of Munich’s Stadtbezirke (districts), we’re able to assign rides to administrative areas and derive deeper insights.

The goal of the analysis is to uncover **meaningful patterns in urban mobility**, understand how users interact with the bike-sharing system, and identify spatial or temporal anomalies.

Each step in the analysis is documented separately, including data cleaning, transformation, visualization, and geospatial reasoning.

---


In [19]:

%useLatestDescriptors
%use dataframe, kandy


# Step 1: Read and Clean Data

This step reads the CSV file using a German locale, then converts time and coordinate fields to usable formats.
It also computes ride durations and a route label combining start and end station names.


In [39]:
import java.util.Locale

val df = DataFrame.readCsv(
    fileOrUrl = "data/mvg-rad-data.csv",
    delimiter = ';',
    parserOptions = ParserOptions(locale = java.util.Locale.GERMAN)
)
df.describe()

name,type,count,unique,nulls,top,freq,mean,std,min,p25,median,p75,max
Row,Int,710106,710106,0,1,1,355053.500000,204990.089464,1,177526.916667,355053.500000,532580.083333,710106
STARTTIME,kotlinx.datetime.LocalDateTime,710106,301480,0,2023-05-19T18:24,25,null,null,2023-01-01T00:26,2023-05-06T16:43,2023-07-04T19:35,2023-09-07T20:35,2023-12-31T23:54
ENDTIME,kotlinx.datetime.LocalDateTime,710106,300963,0,2023-03-09T11:26,22,null,null,2023-01-01T00:42,2023-05-06T17:17,2023-07-04T19:54,2023-09-07T21:04,2024-01-01T16:00
STARTLAT,Double,710106,14295,0,0.000000,7885,47.610000,5.045196,0.000000,48.128250,48.143190,48.159040,53.094660
STARTLON,Double,710106,20472,0,11.558320,3653,11.772959,3.141217,-71.178000,11.549040,11.567820,11.584500,141.353220
ENDLAT,Double,710106,14941,0,0.000000,8103,47.589505,5.170640,-55.973800,48.128300,48.143190,48.159020,53.094660
ENDLON,Double,710106,21391,0,0.000000,3843,11.720168,3.034583,-99.259350,11.549040,11.567710,11.584430,141.353220
RENTAL_IS_STATION,Int?,710106,4,44,0,549834,0.255351,0.739268,0,0.000000,0.000000,0.000000,12
RENTAL_STATION_NAME,String?,710106,332,551795,Sandstraße,3570,null,null,AGROB Nord Ismaning,Giesing,Laimer Platz,Romanplatz,astopark
RETURN_IS_STATION,Int?,710106,4,60,0,583801,0.215537,0.788633,0,0.000000,0.000000,0.000000,12


In [40]:
import org.jetbrains.kotlinx.dataframe.api.*
import java.time.LocalDateTime
import java.time.format.DateTimeFormatter
import java.time.Duration

val formatter = DateTimeFormatter.ofPattern("yyyy-MM-dd'T'HH:mm")


val cleaned = df
    .convert("STARTTIME").with { LocalDateTime.parse(it.toString(), formatter) }
    .convert("ENDTIME").with { LocalDateTime.parse(it.toString(), formatter) }
    .convert("STARTLAT", "STARTLON", "ENDLAT", "ENDLON")
    .with { it.toString().replace(',', '.').toDouble() }

val withDuration = cleaned.add("DURATION_MIN") {
    val start = it["STARTTIME"] as LocalDateTime
    val end = it["ENDTIME"] as LocalDateTime
    Duration.between(start, end).toMinutes().toDouble()
}

val rides = withDuration.add("ROUTE") { row ->
    val start = row["RENTAL_STATION_NAME"]?.toString()?.ifBlank { "Non-station" } ?: "Non-station"
    val end = row["RETURN_STATION_NAME"]?.toString()?.ifBlank { "Non-station" } ?: "Non-station"
    "$start → $end"
}

rides.head()

Row,STARTTIME,ENDTIME,STARTLAT,STARTLON,ENDLAT,ENDLON,RENTAL_IS_STATION,RENTAL_STATION_NAME,RETURN_IS_STATION,RETURN_STATION_NAME,DURATION_MIN,ROUTE
1,2023-01-01T00:26,2023-01-01T00:51,48.137950,11.545690,48.161230,11.557820,0,null,1,Barbarastr,25.000000,Non-station → Barbarastr
2,2023-01-01T00:30,2023-01-01T00:42,48.129030,11.544310,48.147970,11.534450,0,null,0,null,12.000000,Non-station → Non-station
3,2023-01-01T00:32,2023-01-01T00:45,48.168410,11.555660,48.164670,11.576490,0,null,0,null,13.000000,Non-station → Non-station
4,2023-01-01T00:34,2023-01-01T00:46,48.168430,11.555670,48.164640,11.576480,0,null,0,null,12.000000,Non-station → Non-station
5,2023-01-01T00:35,2023-01-01T00:51,48.171040,11.548780,48.162430,11.530070,0,null,0,null,16.000000,Non-station → Non-station


# Step 2: Histogram of Ride Duration

A histogram is plotted for rides under 60 minutes to understand the distribution of trip durations.


In [44]:
import org.jetbrains.letsPlot.Stat

val capped = rides.filter { (it["DURATION_MIN"] as Number).toDouble() < 60 }

capped.plot {
    histogram("DURATION_MIN", binsOption = BinsOption.byNumber(30)) {
        fillColor(Stat.count) {
            scale = continuous(Color.GREEN..Color.RED)
        }
        borderLine.color = Color.BLACK
    }
    layout {
        title = "Duration of bike rides"
        x {
            scale = continuous(limits = 0.0..60.0)
        }
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="SM07Rn"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Duration of bike rides"
},
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"DURATION_MIN",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"scale_mapper_kind":"color_gradient",
"high":"#ee6666",
"low":"#3ba272",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[0.0,60.0]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"count",
"fill":"count"
},
"stat":"identity",
"data":{
"x":[-53.66666666666667,-49.833333333333336,-46.0,-42.16666666666667,-38.333333333333336,-34.5,-30.666666666666668,-26.833333333333336,-23.0,-19.166666666666668,-15.333333333333332,-11.5,-7.666666666666668,-3.833333333333332,0.0,3.833333333333332,7.666666666666668,11.5,15.333333333333336,19.16666666666667,23.0,26.833333333333336,30.66666666666667,34.5,38.333333333333336,42.16666666666667,46.0,49.833333333333336,53.66666666666667,57.5],
"count":[1.0,2.0,3.0,2.0,0.0,0.0,3.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,6188.0,117776.0,169568.0,130911.0,88318.0,56278.0,28460.0,26227.0,17473.0,11683.0,8048.0,5770.0,3350.0,3613.0,2830.0,2305.0]
},
"color":"#000000",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"bar",
"data_meta":{
"series_annotations":[{
"type":"float",
"column":"x"
},{
"type":"int",
"column":"count"
}]
}
}],
"spec_id":"83"
};
 var containerDiv = document.getElementById("SM07Rn");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 
 
 30 
 
 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 
 
 50 
 
 
 
 
 
 
 
 
 60 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 20,000 
 
 
 
 
 
 
 40,000 
 
 
 
 
 
 
 60,000 
 
 
 
 
 
 
 80,000 
 
 
 
 
 
 
 100,000 
 
 
 
 
 
 
 120,000 
 
 
 
 
 
 
 140,000 
 
 
 
 
 
 
 160,000 
 
 
 
 
 
 
 
 
 Duration of bike rides 
 
 
 
 
 count 
 
 
 
 
 DURATION_MIN 
 
 
 
 
 
 
 
 
 count 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 50,000 
 
 
 
 
 
 
 
 
 100,000 
 
 
 
 
 
 
 
 
 150,000

# Step 3: Most Frequent Rental Stations

Top 10 stations where users most frequently rented bikes are shown in a vertical bar chart.


In [23]:
val stationUsage = rides
    .filter { it["RENTAL_STATION_NAME"] != null }
    .groupBy("RENTAL_STATION_NAME")
    .count()
    .sortByDesc("count")
    .take(10)

stationUsage.plot {
    layout.title = "Most Frequent Rental Stations"
    bars {
        x("RENTAL_STATION_NAME") { axis.name = "Station" }
        y("count") { axis.name = "Number of Rentals" }
        fillColor = Color.BLUE
    }
}


<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="bHkEo3"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Most Frequent Rental Stations"
},
"mapping":{
},
"data":{
"RENTAL_STATION_NAME":["Sandstraße","Hauptbahnhof Nord","TUM Arcisstraße","Münchner Freiheit","Universität","Olympiazentrum","Hackerbrücke","Königinstraße","Pasing","Rotkreuzplatz"],
"count":[3570.0,2654.0,2649.0,2428.0,2406.0,2396.0,2188.0,1982.0,1809.0,1794.0]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":"Station"
},{
"aesthetic":"y",
"name":"Number of Rentals",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"RENTAL_STATION_NAME",
"y":"count"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"bar",
"fill":"#5470c6",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"RENTAL_STATION_NAME"
},{
"type":"int",
"column":"count"
}]
},
"spec_id":"41"
};
 var containerDiv = document.getElementById("bHkEo3");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Sandstraße 
 
 
 
 
 
 
 
 
 Hauptbahnhof Nord 
 
 
 
 
 
 
 
 
 TUM Arcisstraße 
 
 
 
 
 
 
 
 
 Münchner Freiheit 
 
 
 
 
 
 
 
 
 Universität 
 
 
 
 
 
 
 
 
 Olympiazentrum 
 
 
 
 
 
 
 
 
 Hackerbrücke 
 
 
 
 
 
 
 
 
 Königinstraße 
 
 
 
 
 
 
 
 
 Pasing 
 
 
 
 
 
 
 
 
 Rotkreuzplatz 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 3,000 
 
 
 
 
 
 
 
 
 Most Frequent Rental Stations 
 
 
 
 
 Number of Rentals 
 
 
 
 
 Station

# Step 4: Top 10 Most Common Routes

The 10 most frequent start→end station routes are plotted as bars, excluding non-station routes.


In [24]:
val routes = rides.groupBy("ROUTE")
    .count()
    .filter { it["ROUTE"] != "Non-station → Non-station" }
    .sortByDesc("count")
    .take(10)

routes.plot {
    bars {
        x("ROUTE") { axis.name = "Route" }
        y("count") { axis.name = "Count" }
        fillColor = Color.ORANGE
    }
    layout.title = "Top 10 Most Common Routes"
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="hAXjxs"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Top 10 Most Common Routes"
},
"mapping":{
},
"data":{
"ROUTE":["Sandstraße → Non-station","Non-station → Sandstraße","Hauptbahnhof Nord → Non-station","TUM Arcisstraße → Non-station","Münchner Freiheit → Non-station","Olympiazentrum → Non-station","Universität → Non-station","Hackerbrücke → Non-station","Non-station → Hauptbahnhof Nord","Non-station → TUM Arcisstraße"],
"count":[2793.0,2267.0,1962.0,1904.0,1894.0,1767.0,1758.0,1731.0,1707.0,1432.0]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":"Route"
},{
"aesthetic":"y",
"name":"Count",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"ROUTE",
"y":"count"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"bar",
"fill":"#fc8452",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"ROUTE"
},{
"type":"int",
"column":"count"
}]
},
"spec_id":"44"
};
 var containerDiv = document.getElementById("hAXjxs");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Sandstraße → Non-station 
 
 
 
 
 
 
 
 
 Non-station → Sandstraße 
 
 
 
 
 
 
 
 
 Hauptbahnhof Nord → Non-station 
 
 
 
 
 
 
 
 
 TUM Arcisstraße → Non-station 
 
 
 
 
 
 
 
 
 Münchner Freiheit → Non-station 
 
 
 
 
 
 
 
 
 Olympiazentrum → Non-station 
 
 
 
 
 
 
 
 
 Universität → Non-station 
 
 
 
 
 
 
 
 
 Hackerbrücke → Non-station 
 
 
 
 
 
 
 
 
 Non-station → Hauptbahnhof Nord 
 
 
 
 
 
 
 
 
 Non-station → TUM Arcisstraße 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 
 
 Top 10 Most Common Routes 
 
 
 
 
 Count 
 
 
 
 
 Route

In [25]:
val routes = rides.groupBy("ROUTE")
    .count()
    .filter { it["ROUTE"]?.toString()?.let { r ->
        !r.startsWith("Non-station") && !r.endsWith("Non-station")
    } == true }
    .sortByDesc("count")
    .take(10)

routes.plot {
    bars {
        x("ROUTE") { axis.name = "Route" }
        y("count") { axis.name = "Count" }
        fillColor = Color.ORANGE
    }
    layout.title = "Top 10 Most Common Routes"
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="wGiz8J"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Top 10 Most Common Routes"
},
"mapping":{
},
"data":{
"ROUTE":["Königinstraße → Königinstraße","U-Bahnhof (P+R) Garching-Hochbrück → Dieselstraße Garching-Hochbrück","Maibaumplatz Garching → Untere Strassäcker Garching","Dieselstraße Garching-Hochbrück → U-Bahnhof (P+R) Garching-Hochbrück","Bahnhof Gräfelfing → Behringstraße Planegg","Olympiazentrum → Olympiazentrum","S-Bahnhof Neugilching → astopark","Eisbachwelle → Eisbachwelle","Chinesischer Turm → Chinesischer Turm","astopark → S-Bahnhof Neugilching"],
"count":[432.0,375.0,371.0,341.0,338.0,321.0,315.0,311.0,302.0,301.0]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":"Route"
},{
"aesthetic":"y",
"name":"Count",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"ROUTE",
"y":"count"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"bar",
"fill":"#fc8452",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"ROUTE"
},{
"type":"int",
"column":"count"
}]
},
"spec_id":"47"
};
 var containerDiv = document.getElementById("wGiz8J");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Königinstraße → Königinstraße 
 
 
 
 
 
 
 
 
 U-Bahnhof (P+R) Garching-Hochbrück → Dieselstraße Garching-Hochbrück 
 
 
 
 
 
 
 
 
 Maibaumplatz Garching → Untere Strassäcker Garching 
 
 
 
 
 
 
 
 
 Dieselstraße Garching-Hochbrück → U-Bahnhof (P+R) Garching-Hochbrück 
 
 
 
 
 
 
 
 
 Bahnhof Gräfelfing → Behringstraße Planegg 
 
 
 
 
 
 
 
 
 Olympiazentrum → Olympiazentrum 
 
 
 
 
 
 
 
 
 S-Bahnhof Neugilching → astopark 
 
 
 
 
 
 
 
 
 Eisbachwelle → Eisbachwelle 
 
 
 
 
 
 
 
 
 Chinesischer Turm → Chinesischer Turm 
 
 
 
 
 
 
 
 
 astopark → S-Bahnhof Neugilching 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 
 
 
 
 
 
 Top 10 Most Common Routes 
 
 
 
 
 Count 
 
 
 
 
 Route

# Step 5: Hourly Usage

A line plot shows ride demand by hour of the day to reveal peak usage times.


In [26]:
import java.time.LocalDateTime

val demandPerHour = rides
    .add("HOUR") { (it["STARTTIME"] as LocalDateTime).hour }
    .groupBy("HOUR")
    .count()
    .sortBy("HOUR")

demandPerHour.plot {
    line {
        x("HOUR") { axis.name = "Hour of Day" }
        y("count") { axis.name = "Number of Rides" }
        color = Color.LIGHT_GREEN
    }
    points {
        x("HOUR")
        y("count")
    }
    layout.title = "Hourly Bike Usage"
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="jAVfyI"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Hourly Bike Usage"
},
"mapping":{
},
"data":{
"HOUR":[0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0],
"count":[12992.0,11002.0,7935.0,5153.0,3254.0,3284.0,8057.0,22984.0,40706.0,30781.0,26650.0,31706.0,39492.0,39900.0,39588.0,42462.0,50280.0,62543.0,66262.0,53352.0,37047.0,28503.0,25741.0,20432.0]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"Hour of Day",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"Number of Rides",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"HOUR",
"y":"count"
},
"stat":"identity",
"color":"#91cc75",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"HOUR",
"y":"count"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"point",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"int",
"column":"HOUR"
},{
"type":"int",
"column":"count"
}]
},
"spec_id":"50"
};
 var containerDiv = document.getElementById("jAVfyI");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M23.319249200605185 261.3080647767672 L23.319249200605185 261.3080647767672 L43.59685720113143 270.5246433353341 L63.87446520165768 284.7292897871555 L84.15207320218393 297.61397398310174 L104.42968120271017 306.40909090909093 L124.70728920323641 306.27014751373565 L144.98489720376267 284.1642533127107 L165.2625052042889 215.0306512304354 L185.54011320481516 132.95215614755992 L205.8177212053414 178.91926277759822 L226.09532920586764 198.0517683180202 L246.37293720639389 174.63517475414378 L266.6505452069201 138.57473221293688 L286.92815320744637 136.6851020361051 L307.2057612079726 138.13011334779998 L327.48336920849886 124.8193360727642 L347.7609772090251 88.61068724317835 L368.03858520955134 31.815258668451918 L388.3161932100776 14.590909090909065 L408.59380121060383 74.38288355879772 L428.87140921113007 149.89861893439218 L449.1490172116563 189.46969793157578 L469.42662521218256 202.2617531972852 L489.7042332127088 226.85010272865784 " fill="none" stroke-width="1.6500000000000001" stroke="rgb(145,204,117)" stroke-opacity="1.0">
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 15 
 
 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 
 
 
 
 10,000 
 
 
 
 
 
 
 20,000 
 
 
 
 
 
 
 30,000 
 
 
 
 
 
 
 40,000 
 
 
 
 
 
 
 50,000 
 
 
 
 
 
 
 60,000 
 
 
 
 
 
 
 
 
 Hourly Bike Usage 
 
 
 
 
 Number of Rides 
 
 
 
 
 Hour of Day

# Step 6: Geolocation of Start and End Points

Using a filtered subset of rides with valid lat/lon, a scatter plot maps the red (start) and blue (end) points.


In [49]:
val sampleSize = 1000
val filteredRides = rides.shuffle().head(1000).filter { row ->
    val startLon = row["STARTLON"] as Double
    val startLat = row["STARTLAT"] as Double
    val endLon = row["ENDLON"] as Double
    val endLat = row["ENDLAT"] as Double

    val lonMin = 11.3
    val lonMax = 11.8
    val latMin = 48.0
    val latMax = 48.3

    val startValid = startLon != 0.0 && startLat != 0.0 &&
            startLon in lonMin..lonMax && startLat in latMin..latMax
    val endValid = endLon != 0.0 && endLat != 0.0 &&
            endLon in lonMin..lonMax && endLat in latMin..latMax

    startValid && endValid
}



filteredRides.plot {
    points {
        x("STARTLON") { axis.name = "Longitude" }
        y("STARTLAT") { axis.name = "Latitude" }
        color = Color.RED
        alpha = 0.5
    }
    points {
        x("ENDLON")
        y("ENDLAT")
        color = Color.BLUE
        alpha = 0.5
    }
    layout.title = "Geolocation of Start (Red) and End (Blue) Points"
}



<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="WZUgrM"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Geolocation of Start (Red) and End (Blue) Points"
},
"mapping":{
},
"data":{
"ENDLON":[11.56676,11.54406,11.57436,11.57136,11.55795,11.55658,11.58006,11.56115,11.56689,11.58905,11.58317,11.55565,11.532,11.56848,11.58065,11.60777,11.586,11.56738,11.56785,11.58653,11.55795,11.56814,11.57567,11.57733,11.60936,11.56169,11.61311,11.55974,11.55847,11.5682,11.60218,11.58863,11.57184,11.5666,11.52555,11.62535,11.58893,11.54874,11.57609,11.55832,11.56654,11.56037,11.57259,11.5407,11.58683,11.50248,11.54919,11.63537,11.56874,11.556,11.53417,11.5375,11.54287,11.59304,11.5876,11.44666,11.59818,11.59981,11.56766,11.55251,11.57736,11.56833,11.52853,11.5735,11.59477,11.55093,11.54359,11.58668,11.56428,11.58409,11.56936,11.53173,11.58934,11.46176,11.53395,11.59436,11.60768,11.55782,11.58874,11.55296,11.53457,11.63137,11.56038,11.53287,11.59435,11.52888,11.56943,11.57302,11.43071,11.562,11.59386,11.55158,11.54962,11.52114,11.56066,11.58199,11.63264,11.54871,11.56472,11.50982,11.57022,11.58888,11.57586,11.57345,11.55151,11.56132,11.52936,11.55942,11.57827,11.59182,11.56533,11.55832,11.56672,11.57776,11.56466,11.55465,11.55978,11.58267,11.57835,11.56085,11.60321,11.53668,11.57428,11.5373,11.57512,11.52078,11.56653,11.58769,11.56263,11.56056,11.5653,11.58175,11.56266,11.67196,11.56204,11.51036,11.56419,11.56604,11.54927,11.57555,11.60046,11.54783,11.54394,11.55955,11.46328,11.56247,11.53436,11.53457,11.56096,11.53567,11.55286,11.55069,11.54904,11.56646,11.61105,11.54775,11.65319,11.53747,11.53437,11.58592,11.55981,11.58654,11.49424,11.55804,11.58694,11.65451,11.57768,11.57799,11.58295,11.55707,11.57563,11.63376,11.56201,11.61994,11.56555,11.61565,11.58777,11.62211,11.56531,11.60836,11.56594,11.54663,11.58924,11.59524,11.57478,11.55535,11.61653,11.58888,11.56073,11.53401,11.5334,11.55611,11.56055,11.61625,11.52517,11.53038,11.57248,11.59773,11.55961,11.44617,11.55455,11.61733,11.58191,11.58168,11.59363,11.50638,11.55366,11.71634,11.56581,11.53471,11.5845,11.53675,11.54453,11.57277,11.57627,11.56377,11.6009,11.57907,11.56448,11.54581,11.56182,11.50353,11.59736,11.56578,11.57514,11.55261,11.56604,11.55311,11.52894,11.57768,11.56136,11.55302,11.50876,11.51253,11.51594,11.55055,11.52834,11.59941,11.56583,11.57144,11.54158,11.58309,11.60403,11.58045,11.63849,11.56227,11.56419,11.59194,11.5427,11.58217,11.58501,11.57013,11.55658,11.59852,11.54202,11.58618,11.58614,11.5202,11.56466,11.60622,11.63392,11.57185,11.54408,11.58064,11.55235,11.53905,11.59019,11.58645,11.54825,11.58008,11.50218,11.62735,11.55784,11.55056,11.56755,11.59258,11.58211,11.5685,11.55857,11.64322,11.56706,11.59736,11.57669,11.57661,11.52912,11.58528,11.54904,11.53713,11.5558,11.55254,11.53753,11.59081,11.56785,11.58266,11.53964,11.56507,11.58796,11.56819,11.42405,11.56668,11.55817,11.56523,11.57438,11.71913,11.54271,11.58684,11.55848,11.56298,11.57029,11.57127,11.55315,11.54201,11.57526,11.56943,11.53299,11.59406,11.54081,11.59412,11.56489,11.57371,11.56851,11.73533,11.60255,11.56729,11.57728,11.51657,11.58654,11.54156,11.51084,11.55646,11.55573,11.55256,11.58549,11.57974,11.5814,11.55553,11.5311,11.52287,11.53134,11.56101,11.58531,11.61163,11.57301,11.6451,11.62582,11.50959,11.54874,11.57282,11.53033,11.553,11.57716,11.56951,11.57296,11.54874,11.5822,11.62757,11.55565,11.5289,11.54218,11.52014,11.56862,11.5583,11.5845,11.57582,11.54052,11.55134,11.5845,11.59587,11.51334,11.58198,11.56564,11.65451,11.52928,11.58564,11.57409,11.53956,11.55975,11.54873,11.56162,11.57152,11.56833,11.59964,11.57974,11.57679,11.54386,11.52704,11.54931,11.59773,11.5583,11.56691,1

# Step 7: Export to GeoJSON

Converts a filtered subset of rides to a GeoJSON format for geospatial analysis or use in map visualizations.


In [28]:
import kotlinx.serialization.Serializable
import kotlinx.serialization.SerialName
import kotlinx.serialization.encodeToString
import kotlinx.serialization.json.Json
import java.io.File
import org.jetbrains.kotlinx.dataframe.DataFrame

@Serializable
@SerialName("FeatureCollection")
data class GeoJsonFeatureCollection(
    val type: String = "FeatureCollection",
    val features: List<GeoJsonFeature>
)

@Serializable
@SerialName("Feature")
data class GeoJsonFeature(
    val type: String = "Feature",
    val geometry: GeoJsonGeometry,
    val properties: Map<String, String>? = null
)

@Serializable
@SerialName("Point")
data class GeoJsonGeometry(
    val type: String = "Point",
    val coordinates: List<Double>
)

fun saveFilteredRidesAsGeoJson(filteredRides: DataFrame<*>, relativeDir: String, filename: String) {
    val features = mutableListOf<GeoJsonFeature>()

    for (row in filteredRides) {
        val startLon = row["STARTLON"] as Double
        val startLat = row["STARTLAT"] as Double
        val endLon = row["ENDLON"] as Double
        val endLat = row["ENDLAT"] as Double

        features.add(
            GeoJsonFeature(
                geometry = GeoJsonGeometry(coordinates = listOf(startLon, startLat)),
                properties = mapOf("point" to "start")
            )
        )
        features.add(
            GeoJsonFeature(
                geometry = GeoJsonGeometry(coordinates = listOf(endLon, endLat)),
                properties = mapOf("point" to "end")
            )
        )
    }

    val featureCollection = GeoJsonFeatureCollection(features = features)
    val jsonString = Json { prettyPrint = true; encodeDefaults = true }.encodeToString(featureCollection)

    val dir = File(relativeDir)
    if (!dir.exists()) dir.mkdirs()  // create folder if it doesn’t exist

    val file = File(dir, filename)
    file.writeText(jsonString)
    println("Saved GeoJSON with ${features.size} points to ${file.absolutePath}")
}

// Save to a relative folder "output" (folder created if missing)
saveFilteredRidesAsGeoJson(filteredRides, "data", "rides.geojson")

// Show preview of saved file
//println(File("output", "rides.geojson").readText().take(500))


Saved GeoJSON with 1926 points to /Users/enriquelopezmanas/Documents/Machine-Learning/mvg-bike-analysis//data/rides.geojson


# Step 8: Map Rides Over Munich Districts

Overlays bike ride start points onto a polygon map of Munich's administrative districts using Kandy-Geo.


In [29]:
%use kandy-geo
val bikeRides =
    GeoDataFrame.readGeoJson("data/rides.geojson")

In [30]:

%use kandy-geo
// Load GeoJSON from your local file path
val munichArea = GeoDataFrame.readGeoJson("data/munich_geojson.json")

munichArea.df.geometry.type().toString()// Plot the polygon(s) of Munich city area
munichArea.plot {
    geoMap() {
        fillColor = Color.LIGHT_BLUE
        borderLine {
            color = Color.RED
            width = 1.5
        }
    }

    layout {
        title = "Munich Metropolitan Area"

    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="bcBrxi"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Munich Metropolitan Area"
},
"mapping":{
},
"coord":{
"name":"map",
"flip":false
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
},
"stat":"identity",
"color":"#ee6666",
"size":1.5,
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"map_data_meta":{
"geodataframe":{
"geometry":"geometry"
}
},
"geom":"polygon",
"fill":"#73c0de",
"map":{
"geometry":["{\"type\":\"Polygon\",\"coordinates\":[[[11.5656601,48.1396451],[11.5737805,48.1434077],[11.5775925,48.1426122],[11.5783692,48.1442706],[11.5834446,48.1431869],[11.585009,48.1469211],[11.5848407,48.1498745],[11.5881922,48.1528542],[11.591757,48.1562371],[11.59408,48.1516584],[11.5981671,48.1493472],[11.5964756,48.1472794],[11.5947898,48.1417059],[11.593937,48.1400224],[11.5901875,48.1372656],[11.586525,48.131746],[11.5818553,48.1336861],[11.5747208,48.1332625],[11.5708224,48.1309851],[11.5682906,48.1314018],[11.5648305,48.1363771],[11.5656601,48.1396451]]]}","{\"type\":\"Polygon\",\"coordinates\":[[[11.586525,48.131746],[11.5836562,48.1283585],[11.5786773,48.1272801],[11.5693244,48.1222148],[11.5625129,48.117857],[11.5441974,48.1263733],[11.5474209,48.1347388],[11.5494795,48.1365705],[11.5497884,48.1388534],[11.5484296,48.1420785],[11.5505423,48.1431655],[11.5607382,48.1411604],[11.5656601,48.1396451],[11.5648305,48.1363771],[11.5682906,48.1314018],[11.5708224,48.1309851],[11.5747208,48.1332625],[11.5818553,48.1336861],[11.586525,48.131746]]]}","{\"type\":\"Polygon\",\"coordinates\":[[[11.5559873,48.1554311],[11.5600412,48.1576365],[11.5706032,48.1565182],[11.5792853,48.1546807],[11.5826803,48.1541495],[11.5817542,48.1524092],[11.5852603,48.1517017],[11.5881922,48.1528542],[11.5848407,48.1498745],[11.585009,48.1469211],[11.5834446,48.1431869],[11.5783692,48.1442706],[11.5775925,48.1426122],[11.5737805,48.1434077],[11.5656601,48.1396451],[11.5607382,48.1411604],[11.5505423,48.1431655],[11.5484296,48.1420785],[11.5389232,48.1427959],[11.5494704,48.1518984],[11.5559873,48.1554311]]]}","{\"type\":\"Polygon\",\"coordinates\":[[[11.5559873,48.1554311],[11.5496316,48.1606824],[11.5529549,48.1614912],[11.5531993,48.1661672],[11.5544974,48.167737],[11.5583709,48.1686096],[11.5615827,48.1684342],[11.5603963,48.174118],[11.5584545,48.1761013],[11.566248,48.1762626],[11.5733961,48.1780331],[11.5852675,48.1776424],[11.5816337,48.1762682],[11.5828975,48.1700616],[11.5826226,48.1671466],[11.5791671,48.1671142],[11.5803956,48.1610771],[11.5792853,48.1546807],[11.5706032,48.1565182],[11.5600412,48.1576365],[11.5559873,48.1554311]]]}","{\"type\":\"Polygon\",\"coordinates\":[[[11.5693244,48.1222148],[11.5786773,48.1272801],[11.5836562,48.1283585],[11.586525,48.131746],[11.5901875,48.1372656],[11.593937,48.1400224],[11.5947898,48.1417059],[11.6065245,48.1391697],[11.6130096,48.1382549],[11.6193453,48.1380893],[11.6186693,48.1350442],[11.6126938,48.1322923],[11.6087116,48.1283615],[11.6033146,48.1247969],[11.6010786,48.1243742],[11.5980486,48.1225959],[11.5770952,48.1185937],[11.5693244,48.1222148]]]}","{\"type\":\"Polygon\",\"coordinates\":[[[11.5625129,48.117857],[11.5609539,48.1161189],[11.5615774,48.1093641],[11.5596875,48.1069933],[11.5551263,48.1046804],[11.5523348,48.1059463],[11.5505252,48.1049515],[11.5441241,48.1054171],[11.5405922,48.1039592],[11.5360485,48.1036113],[11.5358932,48.116642],[11.5376826,48.1246901],[11.5376389,48.1269295],[11.5354245,48.1298569],[11.5404078,48.1282417],[11.5441974,48.1263733],[11.5625129,48.117857]]]}","{\"type\":\"Polygon\",\"coordi

In [31]:

%use kandy-geo
// Load GeoJSON from your local file path
val munichArea = GeoDataFrame.readGeoJson("data/munich_geojson.json")

munichArea.df.geometry.type().toString()// Plot the polygon(s) of Munich city area
munichArea.plot {
    geoMap() {
        fillColor = Color.LIGHT_BLUE
        borderLine {
            color = Color.RED
            width = 1.5
        }
    }
    withData(bikeRides) {
        geoPoints() {
            size = 1.0
            color = Color.YELLOW
        }
    }
    layout {
        title = "Munich Metropolitan Area"
        size = 700 to 500
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="2EUBU5"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Munich Metropolitan Area"
},
"mapping":{
},
"coord":{
"name":"map",
"flip":false
},
"ggsize":{
"width":700.0,
"height":500.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
},
"stat":"identity",
"color":"#ee6666",
"size":1.5,
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"map_data_meta":{
"geodataframe":{
"geometry":"geometry"
}
},
"geom":"polygon",
"fill":"#73c0de",
"map":{
"geometry":["{\"type\":\"Polygon\",\"coordinates\":[[[11.5656601,48.1396451],[11.5737805,48.1434077],[11.5775925,48.1426122],[11.5783692,48.1442706],[11.5834446,48.1431869],[11.585009,48.1469211],[11.5848407,48.1498745],[11.5881922,48.1528542],[11.591757,48.1562371],[11.59408,48.1516584],[11.5981671,48.1493472],[11.5964756,48.1472794],[11.5947898,48.1417059],[11.593937,48.1400224],[11.5901875,48.1372656],[11.586525,48.131746],[11.5818553,48.1336861],[11.5747208,48.1332625],[11.5708224,48.1309851],[11.5682906,48.1314018],[11.5648305,48.1363771],[11.5656601,48.1396451]]]}","{\"type\":\"Polygon\",\"coordinates\":[[[11.586525,48.131746],[11.5836562,48.1283585],[11.5786773,48.1272801],[11.5693244,48.1222148],[11.5625129,48.117857],[11.5441974,48.1263733],[11.5474209,48.1347388],[11.5494795,48.1365705],[11.5497884,48.1388534],[11.5484296,48.1420785],[11.5505423,48.1431655],[11.5607382,48.1411604],[11.5656601,48.1396451],[11.5648305,48.1363771],[11.5682906,48.1314018],[11.5708224,48.1309851],[11.5747208,48.1332625],[11.5818553,48.1336861],[11.586525,48.131746]]]}","{\"type\":\"Polygon\",\"coordinates\":[[[11.5559873,48.1554311],[11.5600412,48.1576365],[11.5706032,48.1565182],[11.5792853,48.1546807],[11.5826803,48.1541495],[11.5817542,48.1524092],[11.5852603,48.1517017],[11.5881922,48.1528542],[11.5848407,48.1498745],[11.585009,48.1469211],[11.5834446,48.1431869],[11.5783692,48.1442706],[11.5775925,48.1426122],[11.5737805,48.1434077],[11.5656601,48.1396451],[11.5607382,48.1411604],[11.5505423,48.1431655],[11.5484296,48.1420785],[11.5389232,48.1427959],[11.5494704,48.1518984],[11.5559873,48.1554311]]]}","{\"type\":\"Polygon\",\"coordinates\":[[[11.5559873,48.1554311],[11.5496316,48.1606824],[11.5529549,48.1614912],[11.5531993,48.1661672],[11.5544974,48.167737],[11.5583709,48.1686096],[11.5615827,48.1684342],[11.5603963,48.174118],[11.5584545,48.1761013],[11.566248,48.1762626],[11.5733961,48.1780331],[11.5852675,48.1776424],[11.5816337,48.1762682],[11.5828975,48.1700616],[11.5826226,48.1671466],[11.5791671,48.1671142],[11.5803956,48.1610771],[11.5792853,48.1546807],[11.5706032,48.1565182],[11.5600412,48.1576365],[11.5559873,48.1554311]]]}","{\"type\":\"Polygon\",\"coordinates\":[[[11.5693244,48.1222148],[11.5786773,48.1272801],[11.5836562,48.1283585],[11.586525,48.131746],[11.5901875,48.1372656],[11.593937,48.1400224],[11.5947898,48.1417059],[11.6065245,48.1391697],[11.6130096,48.1382549],[11.6193453,48.1380893],[11.6186693,48.1350442],[11.6126938,48.1322923],[11.6087116,48.1283615],[11.6033146,48.1247969],[11.6010786,48.1243742],[11.5980486,48.1225959],[11.5770952,48.1185937],[11.5693244,48.1222148]]]}","{\"type\":\"Polygon\",\"coordinates\":[[[11.5625129,48.117857],[11.5609539,48.1161189],[11.5615774,48.1093641],[11.5596875,48.1069933],[11.5551263,48.1046804],[11.5523348,48.1059463],[11.5505252,48.1049515],[11.5441241,48.1054171],[11.5405922,48.1039592],[11.5360485,48.1036113],[11.5358932,48.116642],[11.5376826,48.1246901],[11.5376389,48.1269295],[1

In [32]:
import java.time.DayOfWeek

val ridesWithDayType = rides.add("IS_WEEKEND") { row ->
    val day = (row["STARTTIME"] as LocalDateTime).dayOfWeek
    day == DayOfWeek.SATURDAY || day == DayOfWeek.SUNDAY
}

val usageByDayType = ridesWithDayType
    .groupBy("IS_WEEKEND")
    .count()

usageByDayType.plot {
    bars {
        x("IS_WEEKEND") { axis.name = "Is Weekend" }
        y("count") { axis.name = "Number of Rides" }
        fillColor = Color.LIGHT_BLUE
    }
    layout.title = "Weekend vs Weekday Usage"
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="YCB6kj"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Weekend vs Weekday Usage"
},
"mapping":{
},
"data":{
"count":[203545.0,506561.0],
"IS_WEEKEND":[true,false]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":"Is Weekend"
},{
"aesthetic":"y",
"name":"Number of Rides",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"IS_WEEKEND",
"y":"count"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"bar",
"fill":"#73c0de",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"bool",
"column":"IS_WEEKEND"
},{
"type":"int",
"column":"count"
}]
},
"spec_id":"62"
};
 var containerDiv = document.getElementById("YCB6kj");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 true 
 
 
 
 
 
 
 
 
 false 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 100,000 
 
 
 
 
 
 
 200,000 
 
 
 
 
 
 
 300,000 
 
 
 
 
 
 
 400,000 
 
 
 
 
 
 
 500,000 
 
 
 
 
 
 
 
 
 Weekend vs Weekday Usage 
 
 
 
 
 Number of Rides 
 
 
 
 
 Is Weekend

# Step 9: Rides Per District

Uses JTS to determine which polygon (district) each ride falls into, then counts and visualizes per district.


In [33]:

import org.jetbrains.kotlinx.dataframe.api.*
        import org.locationtech.jts.geom.GeometryFactory
        import org.locationtech.jts.geom.Coordinate
        import org.locationtech.jts.geom.Point
        import org.locationtech.jts.geom.Geometry

// Prepare geometry tools
val geometryFactory = GeometryFactory()

// Extract district name and geometry from munichArea
val districts: List<Pair<String, Geometry>> = munichArea.df.rows().map { row ->
    val name = row["name"]?.toString() ?: "Unknown"  // or use the proper column for district name
    val geometry = row["geometry"] as Geometry
    name to geometry
}

// Assign each ride to a district
val ridesWithDistrict = rides.add("district") { row ->
    val lat = row["STARTLAT"]?.toString()?.toDoubleOrNull()
    val lon = row["STARTLON"]?.toString()?.toDoubleOrNull()

    if (lat == null || lon == null) return@add "Unknown"

    val point = geometryFactory.createPoint(Coordinate(lon, lat))
    val district = districts.firstOrNull { (_, geom) -> geom.contains(point) }

    district?.first ?: "Unknown"
}


// Count rides per district
val rideCounts = ridesWithDistrict
    .groupBy("district")
    .aggregate {
        count().into("rides")
    }
    .sortBy("rides")

val cleanedRideCounts = rideCounts
    .add("district_clean") {
        val regex = Regex("""^Stadtbezirk \d+\s*""")
        regex.replace(it["district"].toString(), "")
    }
cleanedRideCounts.plot {
    layout.title = "Number of Rides per Munich District"
     layout.size = 900 to 500
    barsH {
        y("district_clean") {
            axis.name = "District"
        }
        x("rides") {
            axis.name = "Number of Rides"
        }
        alpha = 0.75

    }
}


<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="b6uLfX"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Number of Rides per Munich District"
},
"mapping":{
},
"data":{
"district_clean":["Allach-Untermenzing","Aubing-Lochhausen-Langwied","Feldmoching-Hasenbergl","Trudering-Riem","Pasing-Obermenzing","Hadern","Thalkirchen-Obersendling-Forstenried-Fürstenried-Solln","Berg am Laim","Ramersdorf - Perlach","Laim","Obergiesing-Fasangarten","Sendling-Westpark","Moosach","Bogenhausen","Untergiesing-Harlaching","Schwanthalerhöhe","Sendling","Milbertshofen-Am Hart","Au-Haidhausen","Unknown","Schwabing-West","Neuhausen-Nymphenburg","Schwabing-Freimann","Altstadt-Lehel","Ludwigsvorstadt-Isarvorstadt","Maxvorstadt"],
"rides":[144.0,155.0,749.0,1474.0,3289.0,3306.0,8401.0,9215.0,9269.0,12833.0,13735.0,15085.0,17664.0,18233.0,18387.0,20251.0,31164.0,34973.0,35325.0,45084.0,48162.0,54174.0,57822.0,64093.0,77187.0,109932.0]
},
"ggsize":{
"width":900.0,
"height":500.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"discrete":true,
"name":"District"
},{
"aesthetic":"x",
"name":"Number of Rides",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"y":"district_clean",
"x":"rides"
},
"stat":"identity",
"orientation":"y",
"sampling":"none",
"alpha":0.75,
"inherit_aes":false,
"position":"dodge",
"geom":"bar",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"district_clean"
},{
"type":"int",
"column":"rides"
}]
},
"spec_id":"65"
};
 var containerDiv = document.getElementById("b6uLfX");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 900.0,
 height: 500.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 20,000 
 
 
 
 
 
 
 
 
 40,000 
 
 
 
 
 
 
 
 
 60,000 
 
 
 
 
 
 
 
 
 80,000 
 
 
 
 
 
 
 
 
 100,000 
 
 
 
 
 
 
 
 
 
 
 Allach-Untermenzing 
 
 
 
 
 
 
 Aubing-Lochhausen-Langwied 
 
 
 
 
 
 
 Feldmoching-Hasenbergl 
 
 
 
 
 
 
 Trudering-Riem 
 
 
 
 
 
 
 Pasing-Obermenzing 
 
 
 
 
 
 
 Hadern 
 
 
 
 
 
 
 Thalkirchen-Obersendling-Forstenried-Fürstenried-Solln 
 
 
 
 
 
 
 Berg am Laim 
 
 
 
 
 
 
 Ramersdorf - Perlach 
 
 
 
 
 
 
 Laim 
 
 
 
 
 
 
 Obergiesing-Fasangarten 
 
 
 
 
 
 
 Sendling-Westpark 
 
 
 
 
 
 
 Moosach 
 
 
 
 
 
 
 Bogenhausen 
 
 
 
 
 
 
 Untergiesing-Harlaching 
 
 
 
 
 
 
 Schwanthalerhöhe 
 
 
 
 
 
 
 Sendling 
 
 
 
 
 
 
 Milbertshofen-Am Hart 
 
 
 
 
 
 
 Au-Haidhausen 
 
 
 
 
 
 
 Unknown 
 
 
 
 
 
 
 Schwabing-West 
 
 
 
 
 
 
 Neuhausen-Nymphenburg 
 
 
 
 
 
 
 Schwabing-Freimann 
 
 
 
 
 
 
 Altstadt-Lehel 
 
 
 
 
 
 
 Ludwigsvorstadt-Isarvorstadt 
 
 
 
 
 
 
 Maxvorstadt 
 
 
 
 
 
 
 
 
 Number of Rides per Munich District 
 
 
 
 
 District 
 
 
 
 
 Number of Rides

# Step 10: Monthly Usage Trend

Converts STARTTIME into year-month format and counts rides per month, plotted as a line chart.


In [34]:
import org.jetbrains.kotlinx.dataframe.api.*
import java.time.LocalDateTime
import java.time.format.DateTimeFormatter

// Parse STARTTIME to LocalDateTime and extract year-month string
val ridesWithMonth = rides.add("yearMonth") { row ->
    val formatter = DateTimeFormatter.ofPattern("yyyy-MM-dd'T'HH:mm")
    val dateTimeStr = row["STARTTIME"].toString()
    val dateTime = LocalDateTime.parse(dateTimeStr, formatter)
    dateTime.format(DateTimeFormatter.ofPattern("yyyy-MM"))
}

// Group by yearMonth and count
val tripsByMonth = ridesWithMonth
    .groupBy("yearMonth")
    .aggregate {
        count().into("trips")
    }
    .sortBy("yearMonth")

// Plot
tripsByMonth.plot {
    layout.title = "Number of Trips by Month"
    line {
        x("yearMonth") {
            axis.name = "Month"
        }
        y("trips") {
            axis.name = "Number of Trips"
        }
        color = Color.BLUE

    }
}


<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="cXveuF"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Number of Trips by Month"
},
"mapping":{
},
"data":{
"yearMonth":["2023-01","2023-02","2023-03","2023-04","2023-05","2023-06","2023-07","2023-08","2023-09","2023-10","2023-11","2023-12"],
"trips":[32630.0,30972.0,55694.0,46392.0,77587.0,100297.0,102861.0,69604.0,70668.0,61531.0,39200.0,22670.0]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":"Month"
},{
"aesthetic":"y",
"name":"Number of Trips",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"yearMonth",
"y":"trips"
},
"stat":"identity",
"color":"#5470c6",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"yearMonth"
},{
"type":"int",
"column":"trips"
}]
},
"spec_id":"68"
};
 var containerDiv = document.getElementById("cXveuF");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M8.860401350137646 248.12150973645382 L8.860401350137646 248.12150973645382 L53.16240810082587 253.66275994463575 L97.4644148515141 171.03864194302326 L141.7664216022023 202.12712773102584 L186.06842835289055 97.8694074389661 L230.37043510357879 21.969653381178716 L274.672441854267 13.400434120757438 L318.9744486049552 124.54962481647891 L363.27645535564346 120.9935993511559 L407.5784621063317 151.53063382169765 L451.88046885701993 226.16372091392353 L496.18247560770817 281.40911653590615 " fill="none" stroke-width="1.6500000000000001" stroke="rgb(84,112,198)" stroke-opacity="1.0">
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2023-01 
 
 
 
 
 
 
 
 
 2023-02 
 
 
 
 
 
 
 
 
 2023-03 
 
 
 
 
 
 
 
 
 2023-04 
 
 
 
 
 
 
 
 
 2023-05 
 
 
 
 
 
 
 
 
 2023-06 
 
 
 
 
 
 
 
 
 2023-07 
 
 
 
 
 
 
 
 
 2023-08 
 
 
 
 
 
 
 
 
 2023-09 
 
 
 
 
 
 
 
 
 2023-10 
 
 
 
 
 
 
 
 
 2023-11 
 
 
 
 
 
 
 
 
 2023-12 
 
 
 
 
 
 
 
 
 
 
 20,000 
 
 
 
 
 
 
 40,000 
 
 
 
 
 
 
 60,000 
 
 
 
 
 
 
 80,000 
 
 
 
 
 
 
 100,000 
 
 
 
 
 
 
 
 
 Number of Trips by Month 
 
 
 
 
 Number of Trips 
 
 
 
 
 Month

# Step 11: Seasonal Usage

Maps each ride to a season based on its month, then counts and plots seasonal usage as horizontal bars.


In [35]:
import org.jetbrains.kotlinx.dataframe.api.*
import java.time.LocalDateTime
import java.time.format.DateTimeFormatter
import org.jetbrains.kotlinx.dataframe.DataRow

val formatter = DateTimeFormatter.ofPattern("yyyy-MM-dd'T'HH:mm")


// Add season column based on STARTTIME month
val ridesWithSeason = rides.add("season") { row ->
    val dateTimeStr = row["STARTTIME"].toString()
    val dateTime = LocalDateTime.parse(dateTimeStr, formatter)
    val month = dateTime.monthValue

    when (month) {
        12, 1, 2 -> "Winter"
        3, 4, 5 -> "Spring"
        6, 7, 8 -> "Summer"
        9, 10, 11 -> "Autumn"
        else -> "Unknown"
    }
}

val seasonOrder = listOf("Spring", "Summer", "Autumn", "Winter")


val tripsBySeason = ridesWithSeason
    .groupBy("season")
    .aggregate {
        count().into("trips")
    }
    // Add a helper column with the season index for sorting
    .add("seasonIndex") { row ->
        seasonOrder.indexOf(row["season"].toString())
    }
    // Sort by that helper column
    .sortBy("seasonIndex")
    // Drop the helper column if you want
    .remove("seasonIndex")

// Reverse the DataFrame order (so Winter is last and appears top)
val reversedTripsBySeason = tripsBySeason.reverse()

reversedTripsBySeason.plot {
    layout.title = "Number of Trips by Season"
    barsH {
        y("season") {
            axis.name = "Season"
        }
        x("trips") {
            axis.name = "Number of Trips"
        }
        alpha = 0.75
        fillColor("season") {
            scale = categoricalColorHue()
        }
    }
}


// Plot horizontal bars by season
reversedTripsBySeason.plot {
    layout.title = "Number of Trips by Season"
    barsH {
        y("season") {
            axis.name = "Season"

        }
        x("trips") {
            axis.name = "Number of Trips"
        }
        alpha = 0.75
        fillColor("season") {
            scale = categoricalColorHue()
        }
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="iP640P"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Number of Trips by Season"
},
"mapping":{
},
"data":{
"trips":[86272.0,171399.0,272762.0,179673.0],
"season":["Winter","Autumn","Summer","Spring"]
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"discrete":true,
"name":"Season"
},{
"aesthetic":"x",
"name":"Number of Trips",
"limits":[null,null]
},{
"aesthetic":"fill",
"scale_mapper_kind":"color_hue"
}],
"layers":[{
"mapping":{
"y":"season",
"x":"trips",
"fill":"season"
},
"stat":"identity",
"orientation":"y",
"sampling":"none",
"alpha":0.75,
"inherit_aes":false,
"position":"dodge",
"geom":"bar",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"season"
},{
"type":"int",
"column":"trips"
}]
},
"spec_id":"71"
};
 var containerDiv = document.getElementById("iP640P");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 50,000 
 
 
 
 
 
 
 
 
 100,000 
 
 
 
 
 
 
 
 
 150,000 
 
 
 
 
 
 
 
 
 200,000 
 
 
 
 
 
 
 
 
 250,000 
 
 
 
 
 
 
 
 
 
 
 Winter 
 
 
 
 
 
 
 Autumn 
 
 
 
 
 
 
 Summer 
 
 
 
 
 
 
 Spring 
 
 
 
 
 
 
 
 
 Number of Trips by Season 
 
 
 
 
 Season 
 
 
 
 
 Number of Trips 
 
 
 
 
 
 
 
 
 season 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Winter 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Autumn 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Summer 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Spring